In [36]:
import pandas as pd
import numpy as np
import torch
import pickle

#Load Static testing and training data
with open('static_data/train.pickle','rb') as handle:
    train_indices = pickle.load(handle)

with open('static_data/test.pickle','rb') as handle:
    test_indices = pickle.load(handle)

In [39]:
df = pd.read_csv("data/chest_xray_train.csv")

In [40]:
# get all pixel columns
selected_features = list(df.columns)
selected_features.remove("file_name")
selected_features.remove("class_id")

# Set train and target
X = df[selected_features]
y = df['class_id']

# Get training and test data
X_train, X_test = X.iloc[train_indices,:], X.iloc[test_indices,:] 
y_train, y_test = y[train_indices], y[test_indices]

In [41]:
X_train, X_test = X.iloc[train_indices,:], X.iloc[test_indices,:] 
y_train, y_test = y[train_indices], y[test_indices]

In [46]:
train_tensor = torch.tensor(X_train.values, dtype= torch.int16)
test_tensor = torch.tensor(y_test.values, dtype = torch.int16)

In [47]:
train_tensor

tensor([[  4,   3,   3,  ...,   0,   0,   0],
        [  0,  13,  45,  ...,   0,   0,   0],
        [ 20,  20,  20,  ...,  20,  20,  20],
        ...,
        [ 92, 102, 179,  ...,  23,  24,  23],
        [ 52,  65,  73,  ...,   0,   0,   0],
        [ 10,  23,  62,  ...,  24,  26,  37]], dtype=torch.int16)

In [48]:
train_tensor.shape

torch.Size([4205, 4096])

In [49]:
test_tensor

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 0, 0, 2, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 0, 2,
        1, 1, 2, 0, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 0, 2, 0, 2, 2, 2, 2, 1,
        1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        2, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 2, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1,
        2, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 1, 1, 2, 2, 1,
        1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 0, 2, 0, 1, 2, 2, 2, 1,
        0, 1, 1, 1, 1, 0, 2, 2, 1, 0, 0, 2, 0, 0, 1, 1, 2, 2, 2, 1, 0, 1, 0, 0,
        1, 0, 2, 0, 2, 0, 0, 0, 1, 2, 1,

In [50]:
test_tensor.shape

torch.Size([467])

In [ ]:
#Custom dataset

In [51]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms.functional import to_pil_image, pil_to_tensor
import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np
from torchvision.io import read_image

from PIL import Image
from PIL import ImageOps

class ChestxrayImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=pil_transform_tensor, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [52]:
img_dir = "C:\\Users\\conor\\OneDrive\\Desktop\\school\\Data Science Minor Independent Study\\chest-xray-research\\archive\\train_images\\train_images"
annotations_file = "C:\\Users\\conor\\OneDrive\\Desktop\\school\\Data Science Minor Independent Study\\chest-xray-research\\archive\\labels_train.csv"

In [56]:
chest_dataset = ChestxrayImageDataset(annotations_file=annotations_file, img_dir=img_dir)

In [58]:
chest_dataset.__getitem__(2)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [1]:
# PIL method
test_img = chest_dataset.__getitem__(2) # get second image in labels list
test_img[0][0].shape # access image in tensor
test_img = test_img[0][0] # image
test_img.shape
test_img = np.array(test_img) # turn to np array
test_pil_img = to_pil_image(test_img) # turn to pil and rescale
img_gray = ImageOps.grayscale(test_pil_img)
image_resized = img_gray.resize((64,64), Image.LANCZOS)
image_resized
image_resized_tensor = pil_to_tensor(image_resized) # turn back into tensor
image_resized_tensor
image_resized_tensor[0]
# plt.figure(figsize=(10,10))
# plt.imshow(image_resized_tensor[0])
# plt.show()

NameError: name 'chest_dataset' is not defined

In [55]:
def pil_transform_tensor(tensor_img):
    # test_img = chest_dataset.__getitem__(2) # get second image in labels list
    # test_img[0][0].shape # access image in tensor
    test_img = test_img[0][0] # image
    # test_img.shape
    test_img = np.array(test_img) # turn to np array
    test_pil_img = to_pil_image(test_img) # turn to pil and rescale
    img_gray = ImageOps.grayscale(test_pil_img)
    image_resized = img_gray.resize((64,64), Image.LANCZOS)
    # image_resized
    image_resized_tensor = pil_to_tensor(image_resized) # turn back into tensor
    # image_resized_tensor
    return image_resized_tensor

torch.Size([1, 64, 64])

In [165]:
#tutorial data loading
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import math
import pickle

class ChestXRayDataset(Dataset):

    def __init__(self):
        # df = pd.read_csv('./data/chest_xray_train.csv')
        
        # #Load Static testing and training data
        
        # with open('static_data/train.pickle','rb') as handle:
        #     train_indices = pickle.load(handle)
        
        # with open('static_data/test.pickle','rb') as handle:
        #     test_indices = pickle.load(handle)

        # # get all pixel columns
        # selected_features = list(df.columns)
        # selected_features.remove("file_name")
        # selected_features.remove("class_id")
        
        # # Set train and target
        # X = df[selected_features]
        # y = df['class_id']
        
        # # Get training and test data
        # X_train, X_test = X.iloc[train_indices,:], X.iloc[test_indices,:] 
        # y_train, y_test = y[train_indices], y[test_indices]

        # # Turn into np arrays 
        # X_train_np = X_train.to_numpy(dtype=np.int16, copy = True)
        # y_test_np = y_test.to_numpy(dtype=np.int16, copy = True)

        # # Turn into PyTorch Sensors
        # X_train_torch = torch.from_numpy(X_train_np)
        # y_test_torch = torch.from_numpy(y_test_np)

        # self.X = X_train_torch
        # self.y = y_test_torch


        df = pd.read_csv('data/chest_xray_train.csv')

        df = df.drop(columns = ["file_name"]) # unnecessary column

        
        columns_len = len(df.columns) # Get number of columns in dataframe
        print(columns_len)
        
        # We have to get the first image in order to initalize array
        first_img = df.iloc[0,0:columns_len-1].to_numpy()
        first_img = np.resize(first_img,(64,64))
        
        # Get first class to initalize array
        second = df.loc[0,"class_id"]
        
        df_X = np.array([first_img],dtype=np.int16) # Image Tensor
        
        df_y = np.array([second],dtype=np.int16) # Class Array
        
        for row in range(1,len(df)):
            
            flattened_img = df.iloc[row,0:columns_len-1].to_numpy() # skip class_id
            class_id = df.loc[row,"class_id"] # get class_id
            matrix_img = np.resize(flattened_img,(64,64)) # turn flattened image back into 2d image
        
        
            df_X = np.append(df_X, [matrix_img],axis = 0) # append image
            df_y = np.append(df_y, class_id) # append class

        X_torch = torch.from_numpy(df_X)
        # y_torch = torch.from_numpy(df_y)

        self.X = X_torch
        # self.y = y_torch
        self.y = df_y

    def __getitem__(self,index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.y.shape[0]

In [10]:
df = pd.read_csv('data/chest_xray_train.csv')

In [12]:
df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel4088,pixel4089,pixel4090,pixel4091,pixel4092,pixel4093,pixel4094,pixel4095,file_name,class_id
0,4,3,3,2,1,0,0,0,0,2,...,108,41,0,0,0,0,0,0,img_1002194571005371555.jpg,1
1,0,13,45,71,86,95,109,103,94,96,...,99,33,0,0,0,0,0,0,img_1002972834724824498.jpg,1
2,20,20,20,20,20,20,20,20,20,20,...,53,13,21,20,20,20,20,20,img_1004160693662088646.jpg,0
3,1,11,27,49,83,107,118,123,131,140,...,149,135,112,99,65,32,10,0,img_1011159426506457600.jpg,2
4,47,77,117,79,140,64,55,62,69,67,...,136,117,68,51,36,17,27,36,img_1014387197248837154.jpg,1


In [13]:
import pickle

#Load Static testing and training data
with open('static_data/train.pickle','rb') as handle:
    train_indices = pickle.load(handle)

with open('static_data/test.pickle','rb') as handle:
    test_indices = pickle.load(handle)

In [14]:
# get all pixel columns
selected_features = list(df.columns)
selected_features.remove("file_name")
selected_features.remove("class_id")

# Set train and target
X = df[selected_features]
y = df['class_id']

# Get training and test data
X_train, X_test = X.iloc[train_indices,:], X.iloc[test_indices,:] 
y_train, y_test = y[train_indices], y[test_indices]

In [25]:
X_train_np = X_train.to_numpy(dtype=np.int16, copy = True)

In [26]:
X_train_np.shape

(4205, 4096)

In [29]:
y_test_np = y_test.to_numpy(dtype=np.int16, copy = True)

In [31]:
y_test_np.shape

(467,)

In [32]:
X_train_torch = torch.from_numpy(X_train_np)

In [34]:
X_train_torch.shape

torch.Size([4205, 4096])

In [35]:
y_test_torch = torch.from_numpy(y_test_np)

In [37]:
y_test_torch.shape

torch.Size([467])

In [42]:
chest_dataset = ChestXRayDataset()

In [46]:
chest_dataset.y.shape

torch.Size([467])

In [ ]:
train_dataloader = DataLoader(dataset = chest_dataset, batch_size = 100, shuffle = True, num_workers = 2)
test_dataloader = DataLoader(dataset = chest_dataset, batch_size = 5000, num_workers = 2)

In [47]:
#Turn pandas df into np array
#dimensions [[64,64],1] [[64x64 image], class_id]
df = pd.read_csv('data/chest_xray_train.csv')

In [50]:
df_np = np.array([],dtype=np.int16)

In [51]:
df_np

array([], dtype=int16)

In [53]:
df = df.drop(columns = ["file_name"]) # unnecessary column

In [67]:
df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel4087,pixel4088,pixel4089,pixel4090,pixel4091,pixel4092,pixel4093,pixel4094,pixel4095,class_id
0,4,3,3,2,1,0,0,0,0,2,...,135,108,41,0,0,0,0,0,0,1
1,0,13,45,71,86,95,109,103,94,96,...,129,99,33,0,0,0,0,0,0,1
2,20,20,20,20,20,20,20,20,20,20,...,110,53,13,21,20,20,20,20,20,0
3,1,11,27,49,83,107,118,123,131,140,...,130,149,135,112,99,65,32,10,0,2
4,47,77,117,79,140,64,55,62,69,67,...,138,136,117,68,51,36,17,27,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4667,203,186,167,63,30,35,41,52,51,50,...,86,95,77,48,17,9,15,16,18,1
4668,88,90,93,103,115,126,121,126,128,129,...,192,186,177,161,124,60,36,48,51,1
4669,92,102,179,120,73,73,83,83,74,67,...,99,51,10,17,22,24,23,24,23,1
4670,52,65,73,81,87,80,104,133,175,203,...,179,170,171,132,93,40,0,0,0,0


In [133]:

columns_len = len(df.columns) # Get number of columns in dataframe
print(columns_len)

# We have to get the first image in order to initalize array
first_img = df.iloc[0,0:columns_len-1].to_numpy()
first_img = np.resize(first_img,(64,64))

# Get first class to initalize array
second = df.loc[0,"class_id"]

df_X = np.array([first_img],dtype=np.int16) # Image Tensor

df_y = np.array([second],dtype=np.int16) # Class Array

for row in range(1,len(df)):
    
    flattened_img = df.iloc[row,0:columns_len-1].to_numpy() # skip class_id
    class_id = df.loc[row,"class_id"] # get class_id
    matrix_img = np.resize(flattened_img,(64,64)) # turn flattened image back into 2d image


    df_X = np.append(df_X, [matrix_img],axis = 0) # append image
    df_y = np.append(df_y, class_id) # append class

    
    

4097


In [134]:
df_X.shape

(4672, 64, 64)

In [136]:
df_y.shape

(4672,)

In [138]:
X_train, X_test = df_X[train_indices,:], df_X[test_indices,:] 
y_train, y_test = df_y[train_indices], df_y[test_indices]

In [139]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(4205, 64, 64) (467, 64, 64) (4205,) (467,)


In [140]:
X_train_torch = torch.from_numpy(X_train)

In [141]:
X_train_torch.shape

torch.Size([4205, 64, 64])

In [142]:
X_test_torch = torch.from_numpy(X_test)
X_test_torch.shape

torch.Size([467, 64, 64])

In [144]:
X_test_torch

tensor([[[ 55,  63,  69,  ..., 153, 165, 138],
         [ 62,  72,  81,  ..., 156, 135, 117],
         [ 60,  80,  89,  ..., 139, 132, 116],
         ...,
         [  0,   0,   0,  ...,   0,   1,   0],
         [  0,   0,   0,  ...,   0,   1,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[ 38,  51,  56,  ...,  66,  46,  30],
         [ 37,  50,  55,  ...,  65,  46,  30],
         [ 35,  51,  58,  ...,  63,  46,  26],
         ...,
         [ 15,  12,  47,  ...,  22,  25,  24],
         [ 15,  12,  46,  ...,  23,  24,  24],
         [ 15,  12,  46,  ...,  23,  24,  25]],

        [[  0,   4,  31,  ...,  31,  19,   6],
         [  0,   2,  26,  ...,  39,  14,   1],
         [  0,   0,  21,  ...,  47,  14,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[ 62,  61,  62,  ..., 131, 157, 179],
         [ 57,  57,  57,  ..., 125, 165, 178]

In [166]:
#Testing New Class
chest_xray = ChestXRayDataset()

4097


In [167]:
len(chest_xray)

4672

In [184]:
chest_xray_train = chest_xray[train_indices]

In [191]:
print(chest_xray_train[0].shape, chest_xray_train[1].shape)

torch.Size([4205, 64, 64]) (4205,)


In [192]:
chest_xray_test = chest_xray[test_indices]

In [193]:
print(chest_xray_test[0].shape, chest_xray_test[1].shape)

torch.Size([467, 64, 64]) (467,)


In [204]:
train_dataloader = DataLoader(dataset = chest_xray, batch_size = 100, shuffle = True, num_workers = 2)

In [205]:
train_dataloader

In [206]:
train_dataiter = iter(train_dataloader)

In [207]:
data = train_dataiter.next()
features,labels = data
print(features, labels)

Exception ignored in: <function _ConnectionBase.__del__ at 0x000001AFFEA9E700>
Traceback (most recent call last):
  File "C:\Users\conor\anaconda3\envs\ImageRecognition\Lib\multiprocessing\connection.py", line 132, in __del__
    self._close()
  File "C:\Users\conor\anaconda3\envs\ImageRecognition\Lib\multiprocessing\connection.py", line 276, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid


AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'